# Spotify charts webscraper

This notebook contains an algorithm that automatically scrapes the Spotify charts website. We will record the absolute streaming count of individual songs (tracks), their title and artist, Track-ID, date and country code. To make sure that the codes we find in the HTML are actually ISO 3166-2 codes we also get a table from Wikipedia and intersect all country codes with the matching ones from the Spotify charts HTML. Did you know which were the top 3 tracks in Germany over the past three years? Hint: the top 1 track was streamed about 143 millions times! <br>
<br>
A separate notebook connects to the Spotify API and joins additional audio features such as danceability, energy, accousticness etc. (something Spotify computes internally) to every track ID. We then identify two strong predictors for popularity of songs by linear regression.

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

# If you get the 'no module named XYZ' error make sure that the package was correctly installed via 'pip install XYZ' in the
# Anaconda prompt (on Windows) and 'sudo easy_install XYZ' in the terminal (on Mac).

At first let's download the HTML file from the Spotify charts page.

In [12]:
spotify_charts = requests.get('https://spotifycharts.com/regional')

In [13]:
spotify_soup = BeautifulSoup(spotify_charts.text, "html5lib")

In [30]:
spotify_soup

<!DOCTYPE html>
<html><head>
        <meta charset="utf-8"/>
        <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
        <title>Spotify Charts</title>

        <meta content="Spotify Charts" property="og:title"/>
        <meta content="website" property="og:type"/>
        <meta content="https://www.spotifycharts.com/" property="og:url"/>
        <meta content="http://d2c87l0yth4zbw-2.global.ssl.fastly.net/i/_global/open-graph-default.png" property="og:image"/>
        <meta content="Spotify is all the music you’ll ever need." property="og:description"/>

        <meta content="Spotify is all the music you’ll ever need." name="description"/>
        <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
        <link href="/images/touch-icon-144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
        <link href="/images/touch-icon-114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
        <link href="/images/touch-icon-72.png" r

We download the ISO 3166-2 country codes from Wikipedia and use the handy pd.read_html function to identify tables.

In [14]:
country_codes_html = requests.get('https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2')

In [39]:
country_codes_df = pd.read_html(country_codes_html.text, flavor ='html5lib', header = 0)[2]

# the [2] accesses the third element inside of the pd.read_html object 
# (so there must be more tables than the one we're interested in)

In [40]:
country_codes_df.head()

,Code,Country name (using title case),Year,ccTLD,ISO 3166-2,Notes
0,AD,Andorra,1974,.ad,ISO 3166-2:AD,NaN
1,AE,United Arab Emirates,1974,.ae,ISO 3166-2:AE,NaN
2,AF,Afghanistan,1974,.af,ISO 3166-2:AF,NaN
3,AG,Antigua and Barbuda,1974,.ag,ISO 3166-2:AG,NaN
4,AI,Anguilla,1985,.ai,ISO 3166-2:AI,AI previously represented French Afars and Issas


In [17]:
country_code_list = list(country_codes_df['Country name (using title case)'].values)

The cell below extracts all elements identified by the selector 'li' from the spotify_soup and appends each element if its text property occurs in the country_code_list we just scraped from Wikipedia to a list named 'spotify_countries'. The sub_df then contains the subset of matching country codes with all available information from Wikipedia.

In [18]:
spotify_countries = []

for i in spotify_soup.find_all('li'):
    
    if i.text in country_code_list:
        
        spotify_countries.append(i.text)

In [19]:
sub_df = country_codes_df[country_codes_df['Country name (using title case)'].isin(spotify_countries)]

In [20]:
spotify_country_codes = list(sub_df['Code'].values)

In [21]:
print("Interesting, Spotify is currently operating in 80 countries but provides charts only for " + str(len(spotify_countries))
      + " countries!")

# https://support.spotify.com/us/article/full-list-of-territories-where-spotify-is-available/

Interesting, Spotify is currently operating in 80 countries but provides charts only for 60 countries!


The cell below searches for elements identified by the selector 'li' in the spotify_soup but now attempts to convert each iterators text property to a pd.to_datetime object. Then the resulting timestamp is converted to a format of 'YYYY-MM-DD'. If the iterator's text property can't be converted to a pd.to_datetime object (i.e. an error would be thrown) the variable 'date' is defined as np.NaN. Finally, all 'date' objects are appended to a list named 'date_list'. In the last line a list comprehension cleans the 'date_list' and keeps only such elements that are not 'nan' and not of type 'float' since they should be of type DateTime.

In [31]:
date_list = []

for i in spotify_soup.find_all('li'):
    
    try:
        
        date = pd.to_datetime(i.text)
        date = date.strftime('%Y-%m-%d')
        
    except:
        
        date = np.NaN
        
    date_list.append(date)
    
date_list_clean = [x for x in date_list if x != 'nan' and type(x) != float]

In [41]:
# We have 1007 actual dates that we're going to retrieve. However, not all dates will be available as the initial response is
# global and may differ by country.

len(date_list_clean)

1007

In [42]:
spotify_country_codes[:10]

['AD', 'AR', 'AT', 'AU', 'BE', 'BG', 'BR', 'CA', 'CH', 'CL']

In [ ]:
seed_url = 'https://spotifycharts.com/regional/'
frequency = 'daily'

country_df_list = []

for country in spotify_country_codes:
    
    date_df_list = []
    
    for date in date_list_clean:
        
        url = seed_url + country.lower() + '/' + frequency + '/' + date # I concatenate the base url with the variable components
                                
        response = requests.get(url) # and send a request for every new URL
        
        if response.status_code == 200: # this line checks whether the response from the request was positive (i.e. code 200)
            
            print("Data available for " + country + ", " + date)
            
            soup_response = BeautifulSoup(response.text, "html5lib")
            response_links = soup_response.find_all('a')
            
            track_links = []
            track_ids = []

            for link in response_links:
    
                if type(link.get('href')) == str and 'track' in link.get('href'):
            
                # this condition checks whether the elements contained in the list 'response_links' which were identified
                # by the selector 'a' are 1) of type string and 2) the link property contains the substring 'track'
                # since I only want entries from the charts website which have a valid track URL
                # Note: there were cases where this wasn't the case, i.e. empty rows or rows without a track URL
                # which led to inconsistencies when merging all dataframes at the end
            
                    track_link = link.get('href')
                    track_id = link.get('href').split('/')[-1]
        
                    track_links.append(track_link)
                    track_ids.append(track_id)
                             
            df = pd.read_html(response.text, flavor ='html5lib', header= 0)[0] # This retrieves the whole table as dataframe
            
            column_names = ['0', 'Rank', '1', 'Title_Artist', 'Streams'] # I assign column names
            df.columns = column_names # and apply the new names to the columns of the dataframe 'df'
            
            na_index = pd.notna(df['Title_Artist']) # I keep only rows where the 'Title_Artist' column is different from
                                                    # nan
            
            na_index_num = [i for i, x in enumerate(na_index) if x]
            
            df = df[pd.notna(df['Title_Artist'])]
            
            # The condition below is necessary whenever there was an invalid (i.e. missing) Track-URL in the table 
            # but the table contained other columns for the associated row.
            
            if len(track_links) != len(df):
                
                df['Track_URL'] = list(np.array(track_links)[na_index_num])
                df['Track_ID'] = list(np.array(track_ids)[na_index_num])
                
                # Now the length of the 'df' and valid 'track_links' indices do match and can be assigned as columns.            
            
            else: 
                
                df['Track_URL'] = track_links
                df['Track_ID'] = track_ids
                
                # For this case no exceptions were needed as the initial table contained as many valid Track-Urls as number 
                # of rows.
            
            df = df[['Rank', 'Title_Artist', 'Streams', 'Track_URL', 'Track_ID']]
            
            # I keep the columns 'Rank', 'Title_Artist', 'Streams', 'Track_URL' and 'Track_ID'
                 
            title_list = []
            artist_list = []
            
            # The loop below splits the concatenated column 'Title_Artist' into 'Title' and 'Artist' at the separator ' by '

            for element in list(df['Title_Artist'].values):
                
                if type(element) == str:
    
                    title = element.split(" by ")[0]
                    artist = element.split(" by ")[1]
    
                    title_list.append(title)
                    artist_list.append(artist)
            
                else:
                    
                    title = np.NaN
                    artist = np.NaN
                    
                    title_list.append(title)
                    artist_list.append(artist)
            
            df['Track title'] = title_list
            df['Artist'] = artist_list
            df['country_code'] = country
            df['date'] = date
            
            date_df_list.append(df)
            
            # Finally the cleaned, resulting daily charts by country dataframe is appended to a list 'date_df_list'
            
            # It is good practice not to send request after request. Depending on the time of execution your program could
            # send too many queries within a period of time and the server could block your IP. 
            # Therefore, include a break of one second by 'time.sleep(1)'
            
#             time.sleep(0.5)
            
        elif response.status_code == 404:
            
            print("No data available for " + country + ", " + date)
            
            date_df_list = []
            
        # The condition below checks whether the 'date_df_list' by country has at least one entry (i.e. day) and if it's true
        # it merges the contained dataframes into one big dataframe and saves it as .csv to your directory.
        # Note that in every iteration the previous table is appended and the previously saved .csv is overwritten to save
        # storage.
        
        if len(date_df_list) > 0:
            
            date_df_merged = pd.concat(date_df_list)
            
#             date_df_merged.to_csv(str(date) + "_" + country + ".csv")

            date_df_merged.to_csv("Latest_" + country + ".csv")
    
    # The line below merges all saved country dataframes into one big dataframe containing daily charts across available
    # countries.
    
    country_df_list.append(date_df_merged)
    
print('Data retrieval finished!')